In [ ]:
# pymatgen == 2022.8.23
# smol == 0.0.5

In [ ]:
import pickle, json, os, random
import numpy as np
import matplotlib.pyplot as plt
import warnings, random
from tqdm import tqdm

from pymatgen.core import Structure, Lattice

from smol.cofe import ClusterSubspace, RegressionData, ClusterExpansion, StructureWrangler
from smol.io import save_work, load_work
from smol.moca import Ensemble, Sampler

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import max_error, r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn.exceptions import ConvergenceWarning

# Generate initial Rocksalt primitive cell

In [ ]:
species = {"Li":0.6, "Mn":0.2, "Ti":0.2}
prim = Structure.from_spacegroup("Fm-3m", Lattice.cubic(4.15213), [species, {"O":1}], [[0,0,0], [0.5, 0.5, 0.5]])

cutoffs = {2:8, 3:6, 4:5, 5:5} 
print(prim.composition)

In [ ]:
prim

In [ ]:
# Or, you can build primitive cell from MnO primitive structure

lat_param = 2.936
cutoffs = {2:8, 3:6, 4:5, 5:5}

prim = Structure.from_file('/home/debbang03/MnO_prim.vasp')
prim.lattice = Lattice.from_parameters(a=lat_param, b=lat_param, c=lat_param,
                                       alpha=60, beta=60, gamma=60)

prim['Mn'] = 'Li0.6Mn0.2Ti0.2'

print(prim.composition)

# Generate cluster subspace

In [ ]:
subspace = ClusterSubspace.from_cutoffs(prim, cutoffs=cutoffs, basis='sinusoid')
print(subspace)

### Load DFT calculation result data used to learning

In [ ]:
file_path = '/home/debbang03/Cluster_expansion/Electrode_engineering/Mn0.4Ti0.4/Data'

with open(file_path + '/stable_300.pkl', 'rb') as f: 
    entry_stable = pickle.load(f)
    
with open(file_path + '/unstable_300.pkl', 'rb') as f: 
    entry_unstable = pickle.load(f)
    
with open(file_path + '/random_300.pkl', 'rb') as f: 
    entry_random = pickle.load(f)

In [ ]:
entry_total = entry_stable + entry_unstable + entry_random
len(entry_total)

In [ ]:
# Plot energy distribution (eV/prim)

energy_stable = [e.energy/32 for e in entry_stable]
energy_unstable = [e.energy/32 for e in entry_unstable]
energy_random = [e.energy/32 for e in entry_random]

plt.figure(figsize=(5,8))

plt.plot([0 for _ in range(len(energy_stable))], energy_stable, 'gx', markersize=15, label='stable')
plt.plot([0 for _ in range(len(energy_unstable))], energy_unstable, 'rx', markersize=15, label='unstable')
plt.plot([0 for _ in range(len(energy_random))], energy_random, 'bx', markersize=15, label='random')

plt.xticks([], [])
plt.ylabel("energy per prim_structure (eV/prim)", fontsize=18)
plt.yticks(fontsize=12)
plt.legend(fontsize=15, frameon=False)

plt.show()
plt.close()

### Add data to StructureWrangler, extract features

In [ ]:
wrangler = StructureWrangler(subspace)
    
for entry in tqdm(entry_total) : 
    wrangler.add_entry(entry)
    
print("Total number of structures used in learning :", wrangler.num_structures)

### Simple linear regression fitting. Check overfitting occurs.

In [ ]:
num_trials = 50

rmse_train_list, rmse_test_list = [], []

for _ in tqdm(range(num_trials)) : 
    x_train, x_test, y_train, y_test = train_test_split(wrangler.feature_matrix, wrangler.get_property_vector(key='energy'), test_size=0.2)
    
    lin_reg = LinearRegression(fit_intercept=False)
    lin_reg.fit(x_train, y_train)
    
    rmse_train = np.sqrt(mse(lin_reg.predict(x_train), y_train))
    rmse_train_list.append(rmse_train)
    
    rmse_test = np.sqrt(mse(lin_reg.predict(x_test), y_test))
    rmse_test_list.append(rmse_test)
    
print("Average RMSE for train : {} (eV/prim)".format(np.mean(rmse_train_list)))
print("Average RMSE for test : {} (eV/prim)".format(np.mean(rmse_test_list)))

### Implement L1-regularization if overfitting occurs.

In [ ]:
warnings.filterwarnings('ignore')

total_train, total_test, alp = [], [], []

for alpha in tqdm(np.logspace(-6, -4, 100)) :

    num_trials = 50

    rmse_train_list, rmse_test_list = [], []

    for _ in range(num_trials) : 
        x_train, x_test, y_train, y_test = train_test_split(wrangler.feature_matrix, wrangler.get_property_vector(key='energy'), test_size=0.2)

        lasso = Lasso(alpha = alpha, fit_intercept=True)
        lasso.fit(x_train[:, 1:], y_train)

        lasso_wvec = np.concatenate((np.array([lasso.intercept_]), lasso.coef_), axis=0)

        y_predict_train = np.dot(x_train, lasso_wvec)
        y_predict_test = np.dot(x_test, lasso_wvec)

        rmse_train = np.sqrt(mse(y_predict_train, y_train))
        rmse_train_list.append(rmse_train)

        rmse_test = np.sqrt(mse(y_predict_test, y_test))
        rmse_test_list.append(rmse_test)
    
    total_train.append(np.mean(rmse_train_list))
    total_test.append(np.mean(rmse_test_list))
    alp.append(alpha)

In [ ]:
plt.figure(figsize=(8,6))

plt.xlabel('alpha', fontsize = 20)
plt.xticks(fontsize = 15)
plt.ylabel('average RMSE (eV/prim)',fontsize = 20)
plt.yticks(fontsize = 15)

plt.plot(alp, total_train, 'bo', markersize = 8, label = 'train_RMSE')
plt.plot(alp, total_test, 'ro', markersize = 8, label = 'test_RMSE')

plt.legend(fontsize = 15, frameon = False)
plt.show()
plt.close()

In [ ]:
m = min(total_test)

for i in range(len(total_test)) : 
    if total_test[i] == m : 
        min_index = i
        
print(alp[min_index])

In [ ]:
# optimized alpha
alpha = 1.592282793341094e-06

num_trials = 50

rmse_train_list, rmse_test_list = [], []

for _ in range(num_trials) : 
    x_train, x_test, y_train, y_test = train_test_split(wrangler.feature_matrix, wrangler.get_property_vector(key='energy'), test_size=0.2)
    
    lasso = Lasso(alpha = alpha, fit_intercept=True)
    lasso.fit(x_train[:, 1:], y_train)
    
    lasso_wvec = np.concatenate((np.array([lasso.intercept_]), lasso.coef_), axis=0)
    
    y_predict_train = np.dot(x_train, lasso_wvec)
    y_predict_test = np.dot(x_test, lasso_wvec)
        
    rmse_train = np.sqrt(mse(y_predict_train, y_train))
    rmse_train_list.append(rmse_train)
    
    rmse_test = np.sqrt(mse(y_predict_test, y_test))
    rmse_test_list.append(rmse_test)
    
print("Average RMSE for train : {} (eV/prim)".format(np.mean(rmse_train_list)))
print("Average RMSE for test : {} (eV/prim)".format(np.mean(rmse_test_list)))

In [ ]:
cation = prim.composition['O'] # Rocksalt composition
rmse_per_cation = 1000 * np.mean(rmse_test_list)/cation

print("Average RMSE for test : {} (meV/cation)".format(round(rmse_per_cation, 2)))

### Train cluster expansion model with Lasso

In [ ]:
alpha = 1.592282793341094e-06
lasso = Lasso(alpha = alpha, fit_intercept=True)
x_data = wrangler.feature_matrix
y_data = wrangler.get_property_vector(key='energy')

lasso.fit(x_data[:, 1:], y_data)
lasso_wvec = np.concatenate((np.array([lasso.intercept_]), lasso.coef_), axis=0)

lasso_reg_data = RegressionData.from_sklearn(lasso, 
                                             wrangler.feature_matrix, 
                                             wrangler.get_property_vector(key='energy'))

lasso_expansion = ClusterExpansion(subspace, 
                                   coefficients=lasso_wvec, 
                                   regression_data = lasso_reg_data)

In [ ]:
file_path = '/home/debbang03/Cluster_expansion/Electrode_engineering/Mn0.4Ti0.4_lasso_CE_test.mson'

save_work(file_path, wrangler, lasso_expansion)

In [ ]:
file_path = '/home/debbang03/Cluster_expansion/Electrode_engineering/Mn0.4Ti0.4_lasso_CE_test.mson'

work = load_work(file_path)

print(work['StructureWrangler'])
print("")
print(work['ClusterExpansion'])

In [ ]:
lasso_expansion = work['ClusterExpansion']

# Plot training accuracy

In [ ]:
pred, dft_energy = [], []
for i in tqdm(range(len(entry_total))) : 
    try : 
        pred.append(lasso_expansion.predict(entry_total[i].structure))
        dft_energy.append(entry_total[i].energy)
    except : 
        pass

In [ ]:
plt.figure(figsize=(6,6))
lim = [-435, -405]

plt.plot(pred, dft_energy, 'o')
plt.xlabel('Predicted energy (eV)', fontsize = 15)
plt.ylabel('DFT energy (eV)', fontsize = 15)
plt.xticks(np.arange(-440, -404, 5), fontsize = 12)
plt.yticks(np.arange(lim[0], lim[1]+1, 5), fontsize = 12)

plt.plot([i for i in range(-440, -404)], [i for i in range(-440, -404)], linestyle = '--', color='gray', alpha = 0.5)

plt.xlim(lim)
plt.ylim(lim)

plt.show()
plt.close()

# Construct canonical ensemble from cluster expansion model

In [ ]:
s = Structure.from_file('/home/debbang03/HYUNDAI_DRX/drx_st/MnO_8x8x8_supercell.vasp')
s.composition

In [ ]:
# Initial random seed

mn = [i for i in range(len(s)) if s[i].specie.name == 'Mn']

li = random.sample(mn, 308)
ti = random.sample([i for i in mn if i not in li], 102)

for i in li : 
    s[i] = 'Li'
for i in ti : 
    s[i] = 'Ti'
    
s.composition

In [ ]:
# MCMC simulation, construct canonical ensemble

mat = lasso_expansion.cluster_subspace.scmatrix_from_structure(s)
ensemble = Ensemble.from_cluster_expansion(lasso_expansion, mat)

init_occu = ensemble.processor.occupancy_from_structure(s)

sampler = Sampler.from_ensemble(ensemble = ensemble, kernel_type = 'Metropolis', temperature=3273)
sampler.run(nsteps = 11000000, initial_occupancies=init_occu, thin_by=10000, progress=True)

In [ ]:
sampler.samples.num_samples

In [ ]:
# Discard initial burn-in
container = sampler.samples.get_occupancies(discard=100)
sampling_data = {}
for i in tqdm(range(len(container))) : 
    oc = container[i]
    st = ensemble.processor.structure_from_occupancy(oc)
    st = st.get_sorted_structure()
    sampling_data['{}'.format(i+1)] = st
    
with open('/home/debbang03/test/3273K_10000thin.pkl', 'wb') as file : 
    pickle.dump(sampling_data, file)